In [ ]:
%pip install chromadb

     ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 19.9/19.9 MB 72.3 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 278.2/278.2 kB 25.6 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 2.0/2.0 MB 55.6 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 103.3/103.3 kB 9.6 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚î

In [ ]:
# app.py
import os
import openai
import chromadb
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv
import time
import re

In [ ]:
class Chatbot:
    """Encapsulates the OpenAI client and response generation logic with Query Expansion."""
    def __init__(self, api_key, model="gpt-4o-mini", base_url="https://api.tapsage.com/openai/v1"):
        if not api_key:
            raise ValueError("OpenAI API key is required.")
        self.model = model
        self.client = openai.OpenAI(api_key=api_key, base_url=base_url)

    def expand_query(self, query):
        """ Uses the LLM to correct potential spelling or grammatical errors in the query.
        Returns exactly two queries: [original, corrected] (if corrected is different).
        """
        if not self.client:
            print("OpenAI client not initialized. Skipping query expansion.")
            return [query, query]

        correction_prompt = f"""
        ÿ¥ŸÖÿß €å⁄© Ÿà€åÿ±ÿß€åÿ¥⁄Øÿ± ÿ≤ÿ®ÿßŸÜ ŸÅÿßÿ±ÿ≥€å Ÿáÿ≥ÿ™€åÿØ. Ÿàÿ∏€åŸÅŸá ÿ¥ŸÖÿß ÿß€åŸÜ ÿßÿ≥ÿ™ ⁄©Ÿá ŸÅŸÇÿ∑ Ÿà ŸÅŸÇÿ∑ ÿßÿ¥ÿ™ÿ®ÿßŸáÿßÿ™ ÿßŸÖŸÑÿß€å€åÿå ŸÜ⁄Øÿßÿ±ÿ¥€å €åÿß ÿØÿ≥ÿ™Ÿàÿ±€å ÿ≥ŸàÿßŸÑ ÿ≤€åÿ± ÿ±ÿß ÿßÿµŸÑÿßÿ≠ ⁄©ŸÜ€åÿØ.
        - ÿß⁄Øÿ± ÿ®Ÿá ÿµŸàÿ±ÿ™ ÿ±ÿ≥ŸÖ€å ŸÜŸàÿ¥ÿ™Ÿá ŸÜÿ¥ÿØŸá ÿ¢ŸÜ ÿ±ÿß ÿ®Ÿá ÿµŸàÿ±ÿ™ ÿ±ÿ≥ŸÖ€å ÿ®ÿßÿ≤ŸÜŸà€åÿ≥€å ⁄©ŸÜ€åÿØ.
        - ÿß⁄Øÿ± ÿ≥ŸàÿßŸÑ ÿßÿ≤ ŸÜÿ∏ÿ± ÿ≤ÿ®ÿßŸÜ€å ÿµÿ≠€åÿ≠ ÿßÿ≥ÿ™ÿå ŸáŸÖÿßŸÜ ÿ≥ŸàÿßŸÑ ÿ±ÿß ÿ®ÿØŸàŸÜ Ÿá€å⁄Ü ÿ™ÿ∫€å€åÿ±€å ÿ®ÿ±⁄Øÿ±ÿØÿßŸÜ€åÿØ.
        - Ÿá€å⁄Ü ÿ®ÿßÿ≤ŸÜŸà€åÿ≥€å ŸÖÿπŸÜÿßÿØÿßÿ±ÿå ÿ™ÿ∫€å€åÿ± ÿ≥ÿ®⁄© €åÿß ÿ¨ÿß€å⁄Øÿ≤€åŸÜ€å Ÿàÿß⁄òŸá ÿßŸÜÿ¨ÿßŸÖ ŸÜÿØŸá€åÿØ.
        - ŸÅŸÇÿ∑ €å⁄© ÿÆÿ∑ Ÿæÿßÿ≥ÿÆ ÿ®ÿØŸá€åÿØ Ÿà Ÿá€å⁄Ü ÿ™Ÿàÿ∂€åÿ≠ ÿßÿ∂ÿßŸÅŸá ŸÜŸÜŸà€åÿ≥€åÿØ.

        ÿ≥ŸàÿßŸÑ: "{query}"
        """

        try:
            print("\n-> Correcting query for grammar/spelling...")
            completion = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "ÿ¥ŸÖÿß €å⁄© Ÿà€åÿ±ÿß€åÿ¥⁄Øÿ± ÿØŸÇ€åŸÇ ÿ≤ÿ®ÿßŸÜ ŸÅÿßÿ±ÿ≥€å Ÿáÿ≥ÿ™€åÿØ."},
                    {"role": "user", "content": correction_prompt}
                ],
                temperature=0.0,  # deterministic output
            )
            corrected = completion.choices[0].message.content.strip()

            # Ensure we have a valid string
            if not corrected:
                corrected = query

            # Return exactly two queries: original + corrected (even if same)
            queries = [query, corrected]
            print(f"   Original: '{query}'")
            print(f"   Corrected: '{corrected}'")
            return queries

        except Exception as e:
            print(f"   Error during query correction: {e}")
            return [query]  # fallback: duplicate original


    def retrieve_relevant_chunks(self, queries, collection, embedding_model, n_results=10):
        """
        Retrieves the top N unique, most relevant chunks from ChromaDB for a list of queries.
        """
        all_retrieved_docs = {}

        print("-> Retrieving relevant chunks...")
        for q in queries:
            query_embedding = embedding_model.encode(q).tolist()
            results = collection.query(
                query_embeddings=[query_embedding],
                n_results=n_results
            )

            for doc in results['documents'][0]:
                if doc not in all_retrieved_docs:
                    all_retrieved_docs[doc] = True

        retrieved_texts = list(all_retrieved_docs.keys())
        print(f"   Retrieved {len(retrieved_texts)} unique chunks.")
        return retrieved_texts

    def generate_response(self, query, context_chunks):
        """
        Generates a final response using the LLM based on the query and retrieved context.
        """
        if not self.client:
            return "OpenAI client is not initialized. Cannot generate response."

        if not context_chunks:
            return "ŸÖÿ™ÿßÿ≥ŸÅÿßŸÜŸá ÿßÿ∑ŸÑÿßÿπÿßÿ™ ⁄©ÿßŸÅ€å ÿ®ÿ±ÿß€å Ÿæÿßÿ≥ÿÆ ÿ®Ÿá ÿ≥ŸàÿßŸÑ ÿ¥ŸÖÿß ÿØÿ± Ÿæÿß€å⁄ØÿßŸá ÿØÿßŸÜÿ¥ ŸÖŸÜ Ÿàÿ¨ŸàÿØ ŸÜÿØÿßÿ±ÿØ."

        # üîç ŸÜŸÖÿß€åÿ¥ chunkŸáÿß€å ÿ®ÿßÿ≤€åÿßÿ®€å‚Äåÿ¥ÿØŸá ÿ®ÿ±ÿß€å ÿØ€åÿ®ÿß⁄Ø/ÿ¥ŸÅÿßŸÅ€åÿ™
        print("\n--- Retrieved Knowledge Chunks (for context) ---")
        for i, chunk in enumerate(context_chunks, 1):
            print(f"\n[Chunk {i}]\n{chunk}\n{'-'*50}")
        print("------------------------------------------------\n")

        context_string = "\n\n---\n\n".join(context_chunks)

        system_prompt = """
        ÿ¥ŸÖÿß €å⁄© ÿØÿ≥ÿ™€åÿßÿ± ÿÆÿØŸÖÿßÿ™ ŸÖÿ¥ÿ™ÿ±€å ŸÖÿ™ÿÆÿµÿµ Ÿà ÿØŸàÿ≥ÿ™ÿßŸÜŸá ÿ®ÿ±ÿß€å €å⁄© ÿ¥ÿ±⁄©ÿ™ Ÿáÿ≥ÿ™€åÿØ. ŸÜÿßŸÖ ÿ¥ŸÖÿß ¬´Ÿæÿ¥ÿ™€åÿ®ÿßŸÜ¬ª ÿßÿ≥ÿ™.
        ÿ¥ŸÖÿß ÿ®ÿß€åÿØ ŸÅŸÇÿ∑ Ÿà ŸÅŸÇÿ∑ ÿ®ÿ± ÿßÿ≥ÿßÿ≥ ¬´ÿ≤ŸÖ€åŸÜŸá¬ª ÿßÿ±ÿßÿ¶Ÿá ÿ¥ÿØŸá ÿ®Ÿá ¬´ÿ≥ŸàÿßŸÑ¬ª ⁄©ÿßÿ±ÿ®ÿ± Ÿæÿßÿ≥ÿÆ ÿØŸá€åÿØ.
        ÿßÿ∑ŸÑÿßÿπÿßÿ™€å ÿ±ÿß ÿßÿ≤ ÿÆŸàÿØÿ™ÿßŸÜ ÿßÿ∂ÿßŸÅŸá ŸÜ⁄©ŸÜ€åÿØ. ÿß⁄Øÿ± ÿ≤ŸÖ€åŸÜŸá ÿ®ÿ±ÿß€å Ÿæÿßÿ≥ÿÆ ÿØÿßÿØŸÜ ⁄©ÿßŸÅ€å ŸÜ€åÿ≥ÿ™ÿå
        ŸÖŸàÿØÿ®ÿßŸÜŸá ÿ®⁄ØŸà€å€åÿØ ⁄©Ÿá ÿßÿ∑ŸÑÿßÿπÿßÿ™ ⁄©ÿßŸÅ€å ÿ®ÿ±ÿß€å Ÿæÿßÿ≥ÿÆ ÿ®Ÿá ÿ¢ŸÜ ÿ≥ŸàÿßŸÑ ÿ±ÿß ŸÜÿØÿßÿ±€åÿØ.
        ***ÿ¥ŸÖÿß ÿ®ÿß€åÿØ ŸáŸÖ€åÿ¥Ÿá Ÿà ŸÅŸÇÿ∑ ÿ®Ÿá ÿ≤ÿ®ÿßŸÜ ŸÅÿßÿ±ÿ≥€å Ÿæÿßÿ≥ÿÆ ÿØŸá€åÿØ.***
        """

        user_prompt = f"ÿ≤ŸÖ€åŸÜŸá: \"{context_string}\"\n\nÿ≥ŸàÿßŸÑ: \"{query}\""

        try:
            print("-> Generating final response...")
            start_time = time.time()
            completion = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0.5,
            )
            end_time = time.time()
            print(f"   Generated response in {end_time - start_time:.2f} seconds.")
            return completion.choices[0].message.content
        except openai.APIError as e:
            print(f"   OpenAI API Error: {e}")
            return "ŸÖÿ™ÿßÿ≥ŸÅÿßŸÜŸá ÿØÿ± ÿßÿ±ÿ™ÿ®ÿßÿ∑ ÿ®ÿß ŸáŸàÿ¥ ŸÖÿµŸÜŸàÿπ€å ÿÆÿ∑ÿß€å€å ÿ±ÿÆ ÿØÿßÿØ. ŸÑÿ∑ŸÅÿß ÿØŸàÿ®ÿßÿ±Ÿá ÿ™ŸÑÿßÿ¥ ⁄©ŸÜ€åÿØ."
        except Exception as e:
            print(f"   An unexpected error occurred: {e}")
            return "€å⁄© ÿÆÿ∑ÿß€å ÿ∫€åÿ±ŸÖŸÜÿ™ÿ∏ÿ±Ÿá ÿ±ÿÆ ÿØÿßÿØ. ŸÑÿ∑ŸÅÿß ÿ®ÿß Ÿæÿ¥ÿ™€åÿ®ÿßŸÜ€å ÿ™ŸÖÿßÿ≥ ÿ®⁄Ø€åÿ±€åÿØ."

In [ ]:
def run_chatbot():
    """
    Main function to initialize and run the command-line chatbot.
    """
    print("--- Starting Semantic Search Chatbot (CLI) ---")

    # 1. Load Environment Variables
    # load_dotenv()
    # api_key = os.environ.get("OPENAI_API_KEY")
    api_key = "tpsg-HURAqpPOuLGZtEReVzj3unfTRXGp45o"
    if not api_key:
        print("FATAL: OPENAI_API_KEY not found. Please set it in your .env file.")
        return

    # 2. Load Embedding Model
    print("Loading embedding model...")
    try:
        model_name = "paraphrase-multilingual-mpnet-base-v2"
        embedding_model = SentenceTransformer(model_name)
        print("Embedding model loaded.")
    except Exception as e:
        print(f"FATAL: Could not load embedding model. Error: {e}")
        return

    # 3. Initialize ChromaDB
    print("Initializing ChromaDB client...")
    try:
        client = chromadb.PersistentClient(path="./chroma_db")
        collection_name = "farsi_rag_collection"
        collection = client.get_collection(name=collection_name)
        print("ChromaDB client and collection loaded successfully.")
    except Exception as e:
        print(f"FATAL: Could not connect to ChromaDB. Have you run ingest.py first? Error: {e}")
        return

    # 4. Initialize Chatbot
    try:
        chatbot = Chatbot(api_key=api_key, base_url="https://api.tapsage.com/openai/v1")
        print("Chatbot client initialized.")
    except ValueError as e:
        print(f"FATAL: {e}")
        return
    print("\n--- Chatbot is ready ---")


    print("ÿ®ÿ±ÿß€å ÿÆÿ±Ÿàÿ¨ 'ÿÆÿ±Ÿàÿ¨' ÿ±ÿß ÿ™ÿß€åŸæ ⁄©ŸÜ€åÿØ")
    # 5. Start Chat Loop
    while True:
        user_query = input("ÿ¥ŸÖÿß: ")
        if user_query.lower() in ['ÿÆÿ±Ÿàÿ¨', 'exit']:
            break

        # 1. Expand the user's query
        expanded_queries = chatbot.expand_query(user_query)

        # 2. Retrieve relevant context using all queries
        context_chunks = chatbot.retrieve_relevant_chunks(expanded_queries, collection, embedding_model)

        # 3. Generate the final answer
        answer = chatbot.generate_response(user_query, context_chunks)

        print(f"\nŸæÿ¥ÿ™€åÿ®ÿßŸÜ: {answer}\n")

In [ ]:
run_chatbot()

--- Starting Semantic Search Chatbot (CLI) ---
Loading embedding model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding model loaded.
Initializing ChromaDB client...
ChromaDB client and collection loaded successfully.
Chatbot client initialized.

--- Chatbot is ready ---
ÿ®ÿ±ÿß€å ÿÆÿ±Ÿàÿ¨ 'ÿÆÿ±Ÿàÿ¨' ÿ±ÿß ÿ™ÿß€åŸæ ⁄©ŸÜ€åÿØ
ÿ¥ŸÖÿß: ÿ≥ŸÑÿßŸÖ

-> Correcting query for grammar/spelling...
   Original: 'ÿ≥ŸÑÿßŸÖ'
   Corrected: 'ÿ≥ŸÑÿßŸÖ'
-> Retrieving relevant chunks...
   Retrieved 10 unique chunks.

--- Retrieved Knowledge Chunks (for context) ---

[Chunk 1]
ÿ±ÿßŸá‚ÄåŸáÿß€å ÿ™ŸÖÿßÿ≥ ÿ®ÿß ŸÖÿß ÿ®Ÿá ÿ¥ÿ±ÿ≠ ÿ≤€åÿ± ÿßÿ≥ÿ™:
- ÿ™ŸÑŸÅŸÜ ŸáŸÖÿ±ÿßŸá: €∞€π€π-€∏€±€∏€±-€∏€π€∑€∏
- ÿ™ŸÑŸÅŸÜ ÿ´ÿßÿ®ÿ™: €∞€±€±-€¥€≤€¥€≥-€≥€¥€¥€≥
- ŸÅ⁄©ÿ≥: €∞€±€±-€¥€≤€¥€≥-€≥€¥€¥€¥
- ÿß€åŸÖ€åŸÑ: info@tabenergy.ir
- ÿ¢ÿØÿ±ÿ≥: ŸÖÿßÿ≤ŸÜÿØÿ±ÿßŸÜÿå ÿ≥ŸàÿßÿØ⁄©ŸàŸá ÿ¥ŸÖÿßŸÑ€åÿå ÿ¥Ÿáÿ±⁄© ÿµŸÜÿπÿ™€å ÿ®ÿ¥ŸÑÿå ŸÅÿßÿ≤ €≤ÿå ŸáŸÖÿ™ €∑
--------------------------------------------------

[Chunk 2]
ŸÅÿ±ÿµÿ™€å ÿ®ÿ±ÿß€å ŸáŸÖ⁄©ÿßÿ±€å Ÿæÿß€åÿØÿßÿ± ÿ®ÿß ÿ™ÿßÿ®ÿßŸÜ ÿßŸÜÿ±⁄ò€å: ÿ¥ÿ±⁄©ÿ™ ÿ™ÿßÿ®ÿßŸÜ ÿßŸÜÿ±⁄ò€å ŸÖÿßÿ≤ŸÜÿØÿ≥Ÿà ÿ®ÿß ŸáÿØŸÅ ÿ™Ÿàÿ≥ÿπŸá ÿ≤€åÿ±ÿ≥ÿßÿÆ

KeyboardInterrupt: Interrupted by user